In [ ]:
import pandas as pd
import numpy as np

# leitura dos dados
dados = pd.read_csv("/content/bandaalfaanalise2023semespaco.csv", sep=';', encoding='latin-1')

In [ ]:
dados.head()

,usuario,Canal,Valor,Banda,Etapa,Inconsistencia,QTDCRIT,NATcrit,QTDpergelicit,qtdInconsist
0,72,AF3,"-5,667371975",A,6,1,4,CN,6,1
1,72,F7,"2,725819296",A,6,1,4,CN,6,1
2,72,F3,"-5,568859027",A,6,1,4,CN,6,1
3,72,FC5,"-5,534017306",A,6,1,4,CN,6,1
4,72,T7,"-13,06673307",A,6,1,4,CN,6,1


In [ ]:
dados.dtypes

usuario            int64
Canal             object
Valor             object
Banda             object
Etapa              int64
Inconsistencia     int64
QTDCRIT            int64
NATcrit           object
QTDpergelicit      int64
qtdInconsist       int64
dtype: object

In [ ]:
dados['Valor'] = dados['Valor'].str.replace(',', '.').astype(float)

In [ ]:
dados['Etapa'] = dados['Etapa'].astype(str)

In [ ]:
#dados['Inconsistencia'] = dados['Inconsistencia'].astype(str)

In [ ]:
dados = dados.drop('qtdInconsist', axis=1)

In [ ]:
dados.head()

,usuario,Canal,Valor,Banda,Etapa,Inconsistencia,QTDCRIT,NATcrit,QTDpergelicit
0,72,AF3,-5.667372,A,6,1,4,CN,6
1,72,F7,2.725819,A,6,1,4,CN,6
2,72,F3,-5.568859,A,6,1,4,CN,6
3,72,FC5,-5.534017,A,6,1,4,CN,6
4,72,T7,-13.066733,A,6,1,4,CN,6


In [ ]:
dados.dtypes

usuario             int64
Canal              object
Valor             float64
Banda              object
Etapa              object
Inconsistencia      int64
QTDCRIT             int64
NATcrit            object
QTDpergelicit       int64
dtype: object

In [ ]:
dados = pd.get_dummies(dados, columns=['Canal', 'Banda', 'Etapa', 'NATcrit'])

In [ ]:
dados.head()

,usuario,Valor,Inconsistencia,QTDCRIT,QTDpergelicit,Canal_AF3,Canal_AF4,Canal_F3,Canal_F4,Canal_F7,...,Banda_A,Etapa_1,Etapa_2,Etapa_3,Etapa_4,Etapa_5,Etapa_6,NATcrit_C,NATcrit_CN,NATcrit_N
0,72,-5.667372,1,4,6,1,0,0,0,0,...,1,0,0,0,0,0,1,0,1,0
1,72,2.725819,1,4,6,0,0,0,0,1,...,1,0,0,0,0,0,1,0,1,0
2,72,-5.568859,1,4,6,0,0,1,0,0,...,1,0,0,0,0,0,1,0,1,0
3,72,-5.534017,1,4,6,0,0,0,0,0,...,1,0,0,0,0,0,1,0,1,0
4,72,-13.066733,1,4,6,0,0,0,0,0,...,1,0,0,0,0,0,1,0,1,0


In [ ]:
# 1-RandomForestClassifier

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, confusion_matrix

In [ ]:
dados = dados.drop('usuario', axis=1)

In [ ]:
# dividir os dados em conjunto de treinamento e teste
X = dados.drop(['Inconsistencia'], axis=1)
y = dados['Inconsistencia']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# selecionar modelo
clf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)

# treinar modelo
clf.fit(X_train, y_train)

RandomForestClassifier(max_depth=10, random_state=42)

In [ ]:
# avaliar desempenho do modelo
y_pred = clf.predict(X_test)
y_pred_prob = clf.predict_proba(X_test)[:, 1]

In [ ]:
# avaliar desempenho do modelo
y_pred = clf.predict(X_test)
print("Precisão:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1-score:", f1_score(y_test, y_pred))
print("AUC-ROC:", roc_auc_score(y_test, y_pred_prob))
print("Acurácia:", accuracy_score(y_test, y_pred))
print("Matriz de Confusão:")
print(confusion_matrix(y_test, y_pred))

Precisão: 0.8747591522157996
Recall: 0.9438669438669439
F1-score: 0.9079999999999999
AUC-ROC: 0.9611948248311885
Acurácia: 0.8727524204702628
Matriz de Confusão:
[[177  65]
 [ 27 454]]


In [ ]:
from sklearn.ensemble import RandomForestRegressor

# separar o dataframe em features e target
X = dados.drop(['Inconsistencia'], axis=1)
y = dados['Inconsistencia']

In [ ]:
# treinar o modelo
model = RandomForestRegressor()
model.fit(X, y)

# obter as importâncias das features
importances = model.feature_importances_


In [ ]:
# criar um dataframe com as importâncias das features
feature_importances = pd.DataFrame({'feature': X.columns, 'importance': importances})

# ordenar as features por importância
feature_importances = feature_importances.sort_values('importance', ascending=False)

# imprimir as 10 features mais importantes
print(feature_importances.head(10))

          feature  importance
2   QTDpergelicit    0.384240
1         QTDCRIT    0.299907
0           Valor    0.100185
26      NATcrit_N    0.064700
25     NATcrit_CN    0.031465
22        Etapa_5    0.030443
18        Etapa_1    0.007210
23        Etapa_6    0.006893
21        Etapa_4    0.006023
19        Etapa_2    0.005852


In [ ]:
#  2- KNN KNeighborsClassifier

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
#from sklearn.metrics import accuracy_score, confusion_matrix
#from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score

In [ ]:
# dividir os dados em conjunto de treinamento e teste
X = dados.drop(['Inconsistencia'], axis=1)
y = dados['Inconsistencia']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# selecionar modelo
clf = KNeighborsClassifier(n_neighbors=5)

# treinar modelo
clf.fit(X_train, y_train)

# avaliar desempenho do modelo
y_pred = clf.predict(X_test)
y_pred_prob = clf.predict_proba(X_test)[:, 1]

# avaliar desempenho do modelo

y_pred = clf.predict(X_test)
print("Precisão:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1-score:", f1_score(y_test, y_pred))
print("AUC-ROC:", roc_auc_score(y_test, y_pred_prob))
print("Acurácia:", accuracy_score(y_test, y_pred))
print("Matriz de Confusão:")
print(confusion_matrix(y_test, y_pred))

Precisão: 0.9090909090909091
Recall: 0.9355509355509356
F1-score: 0.9221311475409837
AUC-ROC: 0.954923454923455
Acurácia: 0.8948824343015215
Matriz de Confusão:
[[197  45]
 [ 31 450]]


In [ ]:
# 3- decision tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
# selecionar modelo
clf = DecisionTreeClassifier(random_state=42)

# treinar modelo
clf.fit(X_train, y_train)

DecisionTreeClassifier(random_state=42)

In [ ]:
# avaliar desempenho do modelo
y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
auc_roc = roc_auc_score(y_test, y_pred)

In [ ]:
print("Acurácia:", accuracy)
print("Precisão:", precision)
print("Recall:", recall)
print("F1-score:", f1)
print("AUC-ROC:", auc_roc)
print("Matriz de Confusão:")
print(confusion_matrix(y_test, y_pred))

Acurácia: 0.9308437067773168
Precisão: 0.9480249480249481
Recall: 0.9480249480249481
F1-score: 0.9480249480249481
AUC-ROC: 0.9223595814504907
Matriz de Confusão:
[[217  25]
 [ 25 456]]


In [ ]:
# 4- SVM

In [ ]:
from sklearn.svm import SVC

In [ ]:
# dividir os dados em conjunto de treinamento e teste
X = dados.drop(['Inconsistencia'], axis=1)
y = dados['Inconsistencia']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
#selecionar modelo
clf = SVC(random_state=42)

#treinar modelo
clf.fit(X_train, y_train)

SVC(random_state=42)

In [ ]:
#avaliar desempenho do modelo
y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
auc_roc = roc_auc_score(y_test, y_pred)

print("Acurácia:", accuracy)
print("Precisão:", precision)
print("Recall:", recall)
print("F1-score:", f1)
print("AUC-ROC:", auc_roc)
print("Matriz de Confusão:")
print(confusion_matrix(y_test, y_pred))

Acurácia: 0.6929460580912863
Precisão: 0.6852646638054364
Recall: 0.9958419958419958
F1-score: 0.811864406779661
AUC-ROC: 0.5433755433755434
Matriz de Confusão:
[[ 22 220]
 [  2 479]]


In [ ]:
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [ ]:
# Normalização dos dados
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# Treinamento e avaliação dos modelos com diferentes kernels
for kernel in ['linear', 'poly', 'rbf', 'sigmoid']:
    svm = SVC(kernel=kernel)
    svm.fit(X_train, y_train)
    accuracy = svm.score(X_test, y_test)
    print(f'Kernel: {kernel}, Acurácia: {accuracy}')

Kernel: linear, Acurácia: 0.665283540802213
Kernel: poly, Acurácia: 0.7012448132780082
Kernel: rbf, Acurácia: 0.6929460580912863
Kernel: sigmoid, Acurácia: 0.4979253112033195


In [ ]:
# deeeeeeeep (rede neural densa)

In [ ]:
#modelo de MLP (Perceptron Multicamadas) com duas camadas ocultas (dense layers) com 
#função de ativação ReLU e uma camada de saída com função de ativação sigmoid. 
#Ele foi construído com a biblioteca TensorFlow e treinado com os dados de treinamento usando o
# otimizador 'adam', a função de perda 'binary_crossentropy' e a métrica de precisão (accuracy).

In [ ]:
import tensorflow as tf

In [ ]:
# Dividir os dados em conjuntos de treinamento e teste
train_data = dados.sample(frac=0.8, random_state=0)
test_data = dados.drop(train_data.index)

In [ ]:
# Definir o modelo de rede neural com tensorflow
model = tf.keras.Sequential([
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [ ]:
# Treinar o modelo com os dados de treinamento
train_labels = train_data.pop('Inconsistencia')
test_labels = test_data.pop('Inconsistencia')

history = model.fit(train_data, train_labels, epochs=100,
                    validation_data=(test_data, test_labels))

Epoch 1/100
91/91 [==============================] - 1s 6ms/step - loss: 0.6144 - accuracy: 0.6779 - val_loss: 0.6160 - val_accuracy: 0.6787
Epoch 2/100
91/91 [==============================] - 0s 3ms/step - loss: 0.5910 - accuracy: 0.6907 - val_loss: 0.5727 - val_accuracy: 0.6967
Epoch 3/100
91/91 [==============================] - 0s 4ms/step - loss: 0.5683 - accuracy: 0.6986 - val_loss: 0.5434 - val_accuracy: 0.7091
Epoch 4/100
91/91 [==============================] - 1s 6ms/step - loss: 0.5475 - accuracy: 0.7107 - val_loss: 0.5267 - val_accuracy: 0.7341
Epoch 5/100
91/91 [==============================] - 0s 3ms/step - loss: 0.5274 - accuracy: 0.7239 - val_loss: 0.5146 - val_accuracy: 0.6981
Epoch 6/100
91/91 [==============================] - 0s 3ms/step - loss: 0.4996 - accuracy: 0.7332 - val_loss: 0.4827 - val_accuracy: 0.7396
Epoch 7/100
91/91 [==============================] - 0s 4ms/step - loss: 0.4838 - accuracy: 0.7467 - val_loss: 0.4736 - val_accuracy: 0.7078
Epoch 8/100
9

Esses resultados mostram o desempenho do modelo de rede neural treinado em cada época (epoch) durante o processo de treinamento. O modelo foi treinado por 100 épocas. O treinamento foi feito com 91 amostras por lote (batch), com um tamanho de lote (batch size) padrão de 32 amostras.

A cada época, a perda (loss) e a precisão (accuracy) são calculadas para o conjunto de treinamento e para o conjunto de validação (val_loss e val_accuracy).

No final do treinamento, a precisão (accuracy) do modelo nos dados de treinamento é de 94,98%, o que significa que o modelo classificou corretamente 94,98% das amostras do conjunto de treinamento. A precisão do modelo nos dados de validação (val_accuracy) é de 88,09%, o que significa que o modelo classificou corretamente 88,09% das amostras do conjunto de validação.

A perda (loss) é um indicador de quão bem o modelo está ajustando os dados de treinamento. Uma perda baixa indica que o modelo está ajustando bem os dados de treinamento. No final do treinamento, a perda do modelo nos dados de treinamento é de 0,1201 e a perda do modelo nos dados de validação é de 0,2685.

In [ ]:
dados.head()


,Valor,Inconsistencia,QTDCRIT,QTDpergelicit,Canal_AF3,Canal_AF4,Canal_F3,Canal_F4,Canal_F7,Canal_F8,...,Banda_A,Etapa_1,Etapa_2,Etapa_3,Etapa_4,Etapa_5,Etapa_6,NATcrit_C,NATcrit_CN,NATcrit_N
0,-5.667372,1,4,6,1,0,0,0,0,0,...,1,0,0,0,0,0,1,0,1,0
1,2.725819,1,4,6,0,0,0,0,1,0,...,1,0,0,0,0,0,1,0,1,0
2,-5.568859,1,4,6,0,0,1,0,0,0,...,1,0,0,0,0,0,1,0,1,0
3,-5.534017,1,4,6,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,1,0
4,-13.066733,1,4,6,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,1,0
